## 문제 정의
- int 혹은 float 타입의 범주형 변수는 모델 학습은 되나, 비정상적으로 학습
- 범주형 변수는 반드시 숫자로 변환, 임의로 설정하는 것은 매우 부적절
- 코드화된 변주형 변수도 숫자로 변환해야함

## 범주형 변수 판별
- 범주형 변수는 상태 공간의 크기가 유한한 변수, 도메인이나 변수 상태 공간을 바탕으로 판단
- int 혹은 float 타입으로 정의된 변수가 반드시 연속형이 아닐 수 있다는 것에 주의

### 더미화
- 가장 일반적, 특정값을 취하는지 여부
- 마지막 변수는 변수간 상관성 제거 및 계산량 감소를 위해 제거
- 과하게 많은 변수를 추가해서 차원의 저주 문제로 이어질 수 있음

### 연속형 변수로 치환
- 기존 변수가 가지는 정보가 일부 손실될 수 있고 활용이 어려움
- 차원의 크기가 변하지 않으며 더 효율적인 변수로 변환

##### Series.unique() : 상태 공간을 확인
#### feature_engine.categorical_encoders.OneHotCategoricalEncoder
- 더미화를 하기 위한 함수
- variables : 범주형 변수의 이름 목록(반드시 str타입)
- drop_last : 마지막 더미 변수를 제거할 지를 결정
- top_categories : 한 변수로부터 만드는 더미 변수 개수 설정, 빈도 기준으로 자름
- pandas.get_dummies()가 훨씬 간단하지만, 새로 들어온 데이터에 적용 불가능

In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('./데이터/car-good.csv')
x = df.drop('Class', axis = 1)
y = df['Class']
train_x, test_x, train_y, test_y = train_test_split(x,y)

In [55]:
train_y.value_counts()

negative    629
positive     19
Name: Class, dtype: int64

In [56]:
# 문자 라벨을 숫자로 치환
train_y.replace({'negative':-1, 'positive': 1}, inplace = True)
test_y.replace({'negative':-1, 'positive': 1}, inplace = True)
test_y

390   -1
556   -1
727   -1
102   -1
558   -1
      ..
561   -1
349   -1
184   -1
635   -1
428   -1
Name: Class, Length: 216, dtype: int64

In [57]:
train_x.head()

,Buying,Maint,Doors,Persons,Lug_boot,Safety
693,low,vhigh,4,4,small,low
657,low,vhigh,2,4,small,low
750,low,high,4,4,med,low
51,vhigh,vhigh,4,4,big,low
581,med,med,4,2,med,high


In [58]:
# 숫자가 매우 작음 -> 모든 변수가 범주형임을 확인
for col in train_x.columns:
    print(col, len(train_x[col].unique()))

Buying 4
Maint 4
Doors 3
Persons 2
Lug_boot 3
Safety 3


#### 더미화를 이용한 범주 변수 처리

In [49]:
train_x = train_x.astype(str) 
# 모든 변수가 범주이므로, 더미화를 위해 전부 string 타입으로 변환

In [50]:
from feature_engine.encoding import OneHotEncoder as OHE
dummy_model = OHE(variables = train_x.columns.tolist(),
                 drop_last = True)

dummy_model.fit(train_x)

d_train_x = dummy_model.transform(train_x)
d_test_x = dummy_model.transform(test_x)
train_y

431   -1
615   -1
384   -1
363   -1
479   -1
      ..
524   -1
41    -1
216   -1
482   -1
750   -1
Name: Class, Length: 648, dtype: int64

In [51]:
# 더미화 모델 테스트
from sklearn.neighbors import KNeighborsClassifier as KNN
model = KNN().fit(d_train_x, train_y)
pred_y = model.predict(d_test_x)

from sklearn.metrics import f1_score
f1_score(test_y, pred_y)


0.0

#### 연속형 변수로 치환

In [59]:
train_df = pd.concat([train_x, train_y], axis = 1)
for col in train_x.columns: # 보통은 범주 변수만 순회
    temp_dict = train_df.groupby(col)['Class'].mean().to_dict() # col에 따른 Class 평균
    train_df[col] = train_df[col].replace(temp_dict) # 변수 치환
    test_x[col] = test_x[col].astype(str).replace(temp_dict) # 테스트 데이터도 치환

<ipython-input-59-af3e049eb6c2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x[col] = test_x[col].astype(str).replace(temp_dict) # 테스트 데이터도 치환


In [60]:
train_df.head()

,Buying,Maint,Doors,Persons,Lug_boot,Safety,Class
693,-0.831169,-1.0,-0.945701,-0.885196,-0.944186,-1.000000,-1
657,-0.831169,-1.0,-0.913462,-0.885196,-0.944186,-1.000000,-1
750,-0.831169,-1.0,-0.945701,-0.885196,-0.936652,-1.000000,-1
51,-1.000000,-1.0,-0.945701,-0.885196,-0.943396,-1.000000,-1
581,-0.928994,-0.9,-0.945701,-1.000000,-0.936652,-0.907407,-1


In [61]:
train_x = train_df.drop('Class', axis=1)
train_y = train_df['Class']

In [62]:
# 치환 모델 테스트
from sklearn.neighbors import KNeighborsClassifier as KNN
model = KNN().fit(d_train_x, train_y)
pred_y = model.predict(d_test_x)

from sklearn.metrics import f1_score
f1_score(test_y, pred_y)

0.0